In [174]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(":memory:")  # temporary in-memory DB


In [16]:
with open(r"C:\Users\hamsi\Downloads\restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)


In [18]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


In [26]:
import sqlite3
import pandas as pd

# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Read SQL file
with open(r"C:\Users\hamsi\Downloads\restaurants.sql", "r") as f:
    sql_script = f.read()

# Execute SQL script
conn.executescript(sql_script)


In [30]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.to_csv(r"C:\Users\hamsi\Downloads\restaurants.csv", index=False)


In [32]:
import pandas as pd

orders = pd.read_csv(r"C:\Users\hamsi\Downloads\orders.csv")
users = pd.read_json(r"C:\Users\hamsi\Downloads\users.json")
restaurants = pd.read_csv(r"C:\Users\hamsi\Downloads\restaurants.csv")


In [48]:
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")

gold_df = df[df["membership"] == "Gold"]

print(gold_df.columns)

print(gold_df.columns.tolist())

result = (
    gold_df
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(result)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')
['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [50]:
top_city = result.idxmax()
print("City with highest Gold revenue:", top_city)

City with highest Gold revenue: Chennai


In [54]:

# Average order value by cuisine
result = (
    df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(result)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [56]:
top_cuisine = result.idxmax()
print("Cuisine with highest average order value:", top_cuisine)


Cuisine with highest average order value: Mexican


In [58]:
# Total spend per user
user_total = (
    df
    .groupby("user_id")["total_amount"]
    .sum()
)

# Users who spent more than 1000
users_above_1000 = user_total[user_total > 1000]

# Count of distinct users
count_users = users_above_1000.count()

print("Number of users with total spend > 1000:", count_users)


Number of users with total spend > 1000: 2544


In [60]:
import pandas as pd

# Create rating ranges (bins)
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

df["rating_range"] = pd.cut(
    df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Total revenue by rating range
result = (
    df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(result)


rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64


C:\Users\hamsi\AppData\Local\Temp\ipykernel_9084\412663849.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


In [62]:
top_range = result.idxmax()
print("Rating range with highest total revenue:", top_range)


Rating range with highest total revenue: 4.6 – 5.0


In [64]:
# Filter Gold members
gold_df = df[df["membership"] == "Gold"]

# Average order value by city for Gold members
result = (
    gold_df
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(result)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [66]:
top_city = result.idxmax()
print("City with highest average Gold order value:", top_city)


City with highest average Gold order value: Chennai


In [68]:
# 1️⃣ Distinct restaurants per cuisine
restaurant_count = (
    df
    .groupby("cuisine")["restaurant_id"]
    .nunique()
)

# 2️⃣ Total revenue per cuisine
revenue = (
    df
    .groupby("cuisine")["total_amount"]
    .sum()
)

# Combine both into one DataFrame
summary = pd.DataFrame({
    "distinct_restaurants": restaurant_count,
    "total_revenue": revenue
})

# Sort by few restaurants first, but high revenue
summary = summary.sort_values(
    by=["distinct_restaurants", "total_revenue"],
    ascending=[True, False]
)

print(summary)


         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Italian                   126     2024203.80
Indian                    126     1971412.58
Mexican                   128     2085503.09


In [70]:
# Total number of orders
total_orders = df["order_id"].nunique()

# Number of orders placed by Gold members
gold_orders = df[df["membership"] == "Gold"]["order_id"].nunique()

# Percentage calculation
percentage = round((gold_orders / total_orders) * 100)

print("Percentage of orders by Gold members:", percentage, "%")


Percentage of orders by Gold members: 50 %


In [72]:
print("Total orders:", total_orders)
print("Gold member orders:", gold_orders)


Total orders: 10000
Gold member orders: 4987


In [118]:
import pandas as pd

# Ensure order_date is in datetime format
df["order_date"] = pd.to_datetime(df["order_date"])

# Create quarter column
df["quarter"] = df["order_date"].dt.to_period("Q")

# Total revenue per quarter
quarter_revenue = (
    df
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(quarter_revenue)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64


C:\Users\hamsi\AppData\Local\Temp\ipykernel_9084\3648330139.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [126]:
top_quarter = quarter_revenue.index[0]
print("Quarter with highest revenue:", top_quarter)


Quarter with highest revenue: 2023Q3


In [136]:
# Step 1: Get user_ids of Gold members
gold_user_ids = users.loc[users["membership"] == "Gold", "user_id"]

# Step 2: Count distinct orders placed by Gold users
gold_orders_count = (
    orders.loc[orders["user_id"].isin(gold_user_ids), "order_id"]
    .nunique()
)

print("Total orders placed by Gold members:", gold_orders_count)



Total orders placed by Gold members: 4987


In [142]:
# Step 1: Get user_ids who are from Hyderabad
hyderabad_user_ids = users.loc[users["city"] == "Hyderabad", "user_id"]

# Step 2: Get orders placed by Hyderabad users
hyderabad_orders = orders.loc[
    orders["user_id"].isin(hyderabad_user_ids),
    "total_amount"
]

# Step 3: Calculate total revenue and round
hyderabad_revenue = round(hyderabad_orders.sum())

print("Total revenue from Hyderabad (rounded):", hyderabad_revenue)



Total revenue from Hyderabad (rounded): 1889367


In [144]:
# Number of distinct users who placed at least one order
distinct_users_count = orders["user_id"].nunique()

print("Number of distinct users who placed at least one order:", distinct_users_count)


Number of distinct users who placed at least one order: 2883


In [146]:
# Step 1: Get user_ids of Gold members
gold_user_ids = users.loc[users["membership"] == "Gold", "user_id"]

# Step 2: Get orders placed by Gold users
gold_orders = orders.loc[
    orders["user_id"].isin(gold_user_ids),
    "total_amount"
]

# Step 3: Calculate average order value and round to 2 decimals
avg_order_value_gold = round(gold_orders.mean(), 2)

print("Average order value for Gold members:", avg_order_value_gold)


Average order value for Gold members: 797.15


In [148]:
# Orders placed for restaurants with rating >= 4.5
orders_high_rating = (
    df.loc[df["rating"] >= 4.5, "order_id"]
    .nunique()
)

print("Number of orders placed for restaurants with rating >= 4.5:", orders_high_rating)


Number of orders placed for restaurants with rating >= 4.5: 3374


In [150]:
# Step 1: Get Gold users with their city
gold_users = users.loc[users["membership"] == "Gold", ["user_id", "city"]]

# Step 2: Merge Gold users with orders
gold_orders = orders.merge(gold_users, on="user_id", how="inner")

# Step 3: Find total revenue per city (Gold members only)
city_revenue = (
    gold_orders
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print("Revenue by city (Gold members):")
print(city_revenue)

# Step 4: Identify top revenue city
top_city = city_revenue.index[0]
print("\nTop revenue city among Gold members:", top_city)

# Step 5: Count orders in that top city
orders_in_top_city = (
    gold_orders.loc[gold_orders["city"] == top_city, "order_id"]
    .nunique()
)

print("Number of orders in top revenue city (Gold members):", orders_in_top_city)


Revenue by city (Gold members):
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

Top revenue city among Gold members: Chennai
Number of orders in top revenue city (Gold members): 1337


In [152]:
orders.merge(users, on="user_id", how="left")


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [154]:
import sqlite3
import pandas as pd

# 1. Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# 2. Load the SQL file
with open(r"C:\Users\hamsi\Downloads\restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)

# 3. Read the restaurants table into pandas
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# 4. Display cuisine and rating columns
print(restaurants[["cuisine", "rating"]].head())


   cuisine  rating
0  Chinese     4.8
1   Indian     4.1
2  Mexican     4.3
3  Chinese     4.1
4  Chinese     4.8


In [156]:
len(df)


10000

In [158]:
# Total number of rows in the final merged dataset
total_rows = df.shape[0]

print("Total number of rows in the final merged dataset:", total_rows)


Total number of rows in the final merged dataset: 10000


In [160]:
df = orders.merge(users, on="user_id", how="left")


In [162]:
df[df["membership"].isna()].head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership


In [164]:
df = orders.merge(users, on="user_id", how="left")


In [166]:
print("users columns:", users.columns.tolist())
print("orders columns:", orders.columns.tolist())


users columns: ['user_id', 'name', 'city', 'membership']
orders columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']


In [168]:
df = orders.merge(restaurants, on="restaurant_id", how="left")


In [170]:
# Check restaurant-related columns
print(restaurants.columns.tolist())


['restaurant_id', 'restaurant_name', 'cuisine', 'rating']


In [172]:
df = orders.merge(users, on="user_id", how="left")
